# Cluster Stays

This Notebook clusters all Stays in your MotionTag csv export and clusters the stay points in it. From that you can get interesting insights about which locations you have visited the most in a year. 

But keep in mind that this does not take the time into account. Many points at a short time interval (e.g. on the same day) are not seperated from many points on e.g. different days.

In [ ]:
## Configuration 2
CSV_FILE = "<FILE_NAME>"
YEAR = 2025
## Tweaking:
MAX_DISTANCE_TO_CENTROID = 65
MIN_CLUSTER_SIZE = 4
MIN_SAMPLES = 4
CLUSTER_SELECTION_EPSILON = 100

import numpy as np
import pandas as pd
import datetime as dt
import shapely as shp
import folium as fl
import matplotlib.cm as cm
import matplotlib.colors as mcolors
from geopandas.geodataframe import GeoDataFrame
from shapely.wkb import loads
from sklearn.cluster import HDBSCAN
from pyproj import Geod
from color_markers import ColorIcon

df = pd.read_csv(CSV_FILE, delimiter=";")
df = df.drop(columns=["user_id", "started_at_timezone", "finished_at_timezone", "detected_mode", "confirmed_at", "misdetected_completely", "merged", "created_at", "updated_at", "started_at_in_timezone", "finished_at_in_timezone", "confirmed_at_in_timezone", "created_at_in_timezone", "updated_at_in_timezone", "comment_feedback", "started_on"])

df['geometry'] = df['geometry'].apply(lambda g: loads(g))

gdf = GeoDataFrame(df, geometry="geometry", crs="EPSG:4326")
gdf = gdf.set_index("id")

gdf.started_at = pd.to_datetime(gdf.started_at, format="ISO8601", utc=True)
gdf.finished_at = pd.to_datetime(gdf.finished_at, format="ISO8601", utc=True)

# gdf.head()

In [ ]:
## Filter for relevant year and for Stays only
gdf_stays = gdf.query(f"(started_at.dt.year == {YEAR} or finished_at.dt.year == {YEAR}) and type == 'Stay'")

In [ ]:
## Cluster Points
gdf_proj = gdf_stays.to_crs(epsg=3857)

coords = np.column_stack([
    gdf_proj.geometry.x,
    gdf_proj.geometry.y
])

starttime = dt.datetime.now()

clusterer = HDBSCAN(
    min_cluster_size=MIN_CLUSTER_SIZE,
    min_samples=MIN_SAMPLES,  
    cluster_selection_epsilon=CLUSTER_SELECTION_EPSILON,
    metric='euclidean',
)
labels = clusterer.fit_predict(coords)

print(f"Clustering took {dt.datetime.now() - starttime}")

gdf_stays['cluster'] = labels
gdf_proj["cluster"] = labels

In [ ]:
## Prepare for Clustering Post-Processing
gdf_proj["const"] = 1

gdf_grouped = gdf_proj[gdf_proj.cluster >= 0].dissolve("cluster", aggfunc={"const": "sum", "purpose": "first"})

def find_cluster_radius(value): 
    try:
        return max(p1.distance(p2) for i, p1 in enumerate(value.geoms) for j, p2 in enumerate(value.geoms) if i < j)
    except AttributeError:
        # print(f"Error with {value}")
        return 0

def medoid_point(multipoint):
    if type(multipoint) is shp.Point:
        return multipoint
    coords = np.array([(p.x, p.y) for p in multipoint.geoms])
    dist_matrix = np.linalg.norm(coords[:, None, :] - coords[None, :, :], axis=2)
    medoid_idx = dist_matrix.sum(axis=1).argmin()
    return shp.Point(coords[medoid_idx])

gdf_grouped["cluster_radius_m"] = gdf_grouped['geometry'].apply(find_cluster_radius)
gdf_grouped["centroid"] = gdf_grouped['geometry'].apply(lambda mp: medoid_point(mp))
# gdf_grouped = gdf_grouped.drop(columns=["geometry", "type", "started_at", "finished_at", "length", "mode", "purpose"])
gdf_grouped = gdf_grouped.drop(columns=["geometry","purpose"])

gdf_grouped = gdf_grouped.set_geometry("centroid")
gdf_grouped = gdf_grouped.to_crs(epsg=4326)

gdf_stays = gdf_stays.join(gdf_grouped, on="cluster")

gdf_stays = gdf_stays.rename(columns={"const": "cluster_count"})
# gdf_stays.head()


In [ ]:

## Post-Processing: Clean Clusters 
geod = Geod(ellps="WGS84")

def distance_to_centroid(cluster, centroid, point):
    if cluster == -1 or centroid is None or point is None:
        return None
    
    _, _, distance = geod.inv(centroid.x, centroid.y, point.x, point.y)
    
    return distance

gdf_stays["distance_to_centroid"] = gdf_stays.apply(lambda row: distance_to_centroid(row["cluster"], row["centroid"], row["geometry"]), axis=1)

gdf_stays["invalid_point"] = gdf_stays.apply(lambda row: row["distance_to_centroid"] is None or row["distance_to_centroid"] > MAX_DISTANCE_TO_CENTROID, axis=1)

# gdf_stays.head()

In [ ]:
## Post-Processing: Count Valid points

gdf_only_valid = gdf_stays.query("not invalid_point and cluster != -1")
gdf_only_valid["const"] = 1

gdf_only_valid = gdf_only_valid.dissolve("cluster", aggfunc={"const": "sum"})
gdf_only_valid = gdf_only_valid.drop(columns=["geometry"])
gdf_only_valid = gdf_only_valid.rename(columns={"const": "cluster_count_clean"})
gdf_only_valid.head(n = 25)

gdf_stays = gdf_stays.join(gdf_only_valid, on="cluster")

# gdf_stays.head(n = 20)


In [ ]:
# Post-Processing: Classify Cluster "Quality" (perfect|good|ok|bad)

def determine_cluster_quality(row):
    if row.cluster_count_clean == row.cluster_count: # all points within radius => very dense cluster
        return "perfect"
    if row.cluster_count_clean >= (row.cluster_count * 0.8): # nearly points within radius => very dense cluster
        return "good"
    if row.cluster_count_clean >= (row.cluster_count * 0.5): # most points within radius => ok dense cluster
        return "ok"
    return "bad"

gdf_stays["cluster_quality"] = gdf_stays.apply(lambda row: determine_cluster_quality(row), axis=1)

# gdf_stays.head()

In [ ]:
# gdf_stays_no_noise = gdf_stays.query("not invalid_point and cluster != -1 and cluster_count_clean > 0")
gdf_stays_no_noise = gdf_stays.query("cluster != -1")
gdf_noise = gdf_stays.query("cluster == -1")

gdf_stays_no_noise_grouped = gdf_stays_no_noise.dissolve("cluster")
gdf_stays_no_noise_grouped = gdf_stays_no_noise_grouped.drop(columns=["geometry", "type", "started_at", "finished_at", "length", "mode", "invalid_point", "distance_to_centroid"])
gdf_stays_no_noise_grouped = gdf_stays_no_noise_grouped.sort_values("cluster_count_clean", ascending=False)

m = fl.Map(location=[
    gdf_stays_no_noise.geometry.y.mean(),
    gdf_stays_no_noise.geometry.x.mean()
], zoom_start=6, tiles="CartoDB positron")

noise_layer = fl.FeatureGroup("Noise").add_to(m)
for _, row in gdf_noise.iterrows():
    if row.geometry is None:
        continue
    fl.CircleMarker(
        location=[row.geometry.y, row.geometry.x],
        radius=2,
        fill=True,
        fill_opacity=0.75,
        opacity=1,
        color="light_grey",
        tooltip=fl.Tooltip(
            f"cluster: {row['cluster']}<br>"
            f"started_at: {row["started_at"]}<br>"
            f"purpose: {row["purpose"]}<br>"
            f"id: {row.name}"
        )
    ).add_to(noise_layer)

categories = gdf_stays_no_noise["cluster"].astype(str).unique()
cmap = cm.get_cmap("tab20", len(categories))
color_map = {
    cat: mcolors.to_hex(cmap(i))
    for i, cat in enumerate(categories)
}

fg = {
    "perfect": fl.FeatureGroup("Perfect Clusters").add_to(m),
    "good": fl.FeatureGroup("Good Clusters").add_to(m),
    "ok": fl.FeatureGroup("Ok Clusters").add_to(m),
    "bad": fl.FeatureGroup("Bad Clusters", show=False).add_to(m)
}
quality_colors = {
    "perfect": "black",
    "good": "green",
    "ok": "orange",
    "bad": "red"
}

n = 1
for _, row in gdf_stays_no_noise_grouped.iterrows():
    cluster_count_clean = 0 if str(row["cluster_count_clean"]) == "nan" else row["cluster_count_clean"]
    fl.Circle(
        location=[row.centroid.y, row.centroid.x],
        radius=MAX_DISTANCE_TO_CENTROID,
        color=quality_colors[row["cluster_quality"]],
        weight=2,
        fill_opacity=0.3,
        opacity=1,
        fill_color=color_map[str(row.name)],
        fill=True
    ).add_to(fg[row["cluster_quality"]])

    popup = f"""
    <b>Cluster {row.name}</b><br/>
    Points: <b>{int(cluster_count_clean)}</b>/{int(row.cluster_count)} 
    ({round((cluster_count_clean / row.cluster_count) * 100)}%) <br/>
    Radius: {round(row.cluster_radius_m)}m
    """

    fl.Marker(
        location=[row.centroid.y, row.centroid.x],
        popup=popup,
        icon=ColorIcon(quality_colors[row["cluster_quality"]])
    ).add_to(fg[row["cluster_quality"]])
    n += 1

for _, row in gdf_stays_no_noise.iterrows():
    cluster_count_clean = str(row["cluster_count_clean"])
    fl.CircleMarker(
      location=[row.geometry.y, row.geometry.x],
        radius=2,
        fill=True,
        fill_opacity=0.75,
        opacity=1,
        fill_color=color_map[str(row["cluster"])],
        color=color_map[str(row["cluster"])],
        tooltip=fl.Tooltip(
            f"cluster: {row['cluster']}<br>"
            f"cluster_count_clean: {row['cluster_count_clean']}<br>"
            f"cluster_count: {row['cluster_count']}<br>"
            f"started_at: {row["started_at"]}<br>"
            f"purpose: {row["purpose"]}<br>"
            f"id: {row.name}"
        ),
    ).add_to(fg[row["cluster_quality"]])
    n += 1

fl.LayerControl().add_to(m)

m